<a href="https://colab.research.google.com/github/vintagedeek/mnist/blob/master/kaggle_mnist_feed_forward_no_hidden_no_cnn_2020_07_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np
import os
# df_train = pd.read_csv('sample_data/mnist_train_small.csv') # this abridged mnist dataset is available directly thru google colab

To upload datasets, go to github and click view raw. Then copy the link in the browser and use the url as shown below.

In [49]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [50]:
y = torch.tensor(np.array([1,2,3,6,4,9]), dtype=torch.int)

In [51]:
y

tensor([1, 2, 3, 6, 4, 9], dtype=torch.int32)

In [52]:
z = torch.zeros((6, 10))
z

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [53]:
ind = []
for i in y.numpy():
  ind.append(i)
ind

[1, 2, 3, 6, 4, 9]

In [54]:
for i in range(len(z)):
  z[i][ind[i]] = 1

In [55]:
z

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [56]:
z.dtype

torch.float32

In [57]:
torch.tensor(z, dtype=torch.long).dtype

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.int64

In [58]:
train_url = "https://raw.githubusercontent.com/wehrley/Kaggle-Digit-Recognizer/master/train.csv"
df_train = pd.read_csv(train_url)

In [59]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [60]:
test_url = 'https://raw.githubusercontent.com/wehrley/Kaggle-Digit-Recognizer/master/test.csv'
df_test = pd.read_csv(test_url)

In [61]:
os.listdir('sample_data')

['README.md',
 'anscombe.json',
 'california_housing_test.csv',
 'mnist_train_small.csv',
 'california_housing_train.csv',
 'mnist_test.csv']

In [62]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [63]:
df_test.head()[0:1]

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
df_train.head()[0:1]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


70% of training data in train set and 30% in validation set

In [65]:
42000*.7

29399.999999999996

In [66]:
train_set = df_train[0:29400]
train = torch.tensor(np.array(train_set.iloc[:, 1:]), dtype=torch.torch.float32)
train_labels_raw = torch.tensor(np.array(train_set.iloc[:, 0]), dtype=torch.int64)
val_set = df_train[29400:]
val = torch.tensor(np.array(val_set.iloc[:, 1:]), dtype=torch.float32)
val_labels_raw = torch.tensor(np.array(val_set.iloc[:, 0]), dtype=torch.int64)

In [67]:
train.shape

torch.Size([29400, 784])

In [68]:
train_labels_raw.unique()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [69]:
train_labels = torch.zeros(len(train), 10)
ind = []
for i in train_labels_raw.numpy():
  ind.append(i)

for i in range(len(train_labels)):
  train_labels[i][ind[i]] = 1
train_labels = torch.tensor(train_labels, dtype=torch.int64)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [70]:
train_labels[0]

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [71]:
train_labels_raw[0]

tensor(1)

In [72]:
in_features = train.shape[1]
out_features = 10

In [73]:
print(in_features, out_features)

784 10


In [74]:
x = train[0:2]

In [75]:
x.shape

torch.Size([2, 784])

In [76]:
x = x.reshape(2, 28, 28)

In [77]:
x[0].shape

torch.Size([28, 28])

In [78]:
train.shape

torch.Size([29400, 784])

In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [80]:
device

'cuda'

In [108]:
train_labels.shape

torch.Size([29400, 10])

In [109]:
train.shape

torch.Size([29400, 784])

In [110]:
train_labels[0, 9]

tensor(0)

In [111]:
network = nn.Sequential(
    nn.Linear(in_features=in_features, out_features=out_features),
    nn.Sigmoid())

In [107]:
batch_size = 50
epochs = 70
learning_rate = 0.00001

In [112]:
network = network.to(device)
optimizer = optim.SGD(network.parameters(), lr=learning_rate, weight_decay=10.0)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
for j in range(epochs):
    num_correct = 0
    mini_batches = [train[k:k + batch_size] for k in range(0, train_labels_raw.shape[0], batch_size)]
    mini_labels = [train_labels_raw[k:k + batch_size] for k in range(0, train_labels_raw.shape[0], batch_size)]
    for b in range(0, len(mini_batches)):
        images = mini_batches[b].to(device)
        #print(images.shape)
        labels = mini_labels[b].to(device)
        #print(labels.shape)
        #print(labels.dtype)
        #print(images.dtype)
        preds = network(images)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        num_correct += preds.argmax(dim=1).eq(labels).sum().item()
      
    #scheduler.step()

In [113]:
network[0].weight

Parameter containing:
tensor([[ 5.3128e-04,  5.0548e-04, -3.1614e-04,  ...,  3.7984e-04,
         -5.8051e-04,  5.6146e-04],
        [ 4.5996e-04, -2.6881e-04,  2.2910e-04,  ...,  2.4606e-04,
          4.0803e-04, -2.7848e-04],
        [ 2.0585e-04,  6.7342e-05, -8.7023e-05,  ...,  4.2063e-04,
         -4.8927e-04, -9.9537e-05],
        ...,
        [-4.9609e-04, -2.8406e-04, -2.5250e-04,  ...,  2.1797e-04,
         -4.5940e-04, -3.3797e-04],
        [-5.0946e-04, -5.2009e-04, -2.9394e-04,  ..., -5.3972e-04,
         -2.0368e-04, -3.7911e-05],
        [-2.7028e-04,  4.3031e-04, -8.3530e-05,  ..., -4.7326e-04,
          5.4254e-04, -4.6751e-04]], device='cuda:0', requires_grad=True)

In [114]:
num_correct

26482

In [115]:
len(train)

29400

In [116]:
num_correct/len(train) #  (lr 0.00001) batch=50

0.9007482993197279

In [117]:
## try val set 
val = val.to(device)
val_labels_raw = val_labels_raw.to(device)
val_preds = network(val)
val_num_correct = val_preds.argmax(dim=1).eq(val_labels_raw).sum().item()

In [118]:
val_num_correct/len(val)

0.8958730158730158

In [93]:
val_preds[0]

tensor([0.0834, 0.2577, 0.1464, 0.9327, 0.0356, 0.7745, 0.0404, 0.1584, 0.4480,
        0.2352], device='cuda:0', grad_fn=<SelectBackward>)

In [94]:
val_labels_raw[0]

tensor(3, device='cuda:0')

Increasing learning rate from 0.001 to 0.0001 improved training accuracy from 40% to 79%. Note, played with batch sizes ranging from 20 to 200 or so. Batch size 50 got me from 6000/29400 to 11500/24900. Then played with learning rate. Going from 0.01 to 0.0001 got me from 40 to 79%. Then 0.00001 got me to 82%. Reducing the learning rate by another factor of 10 (0.000001) reduced accuracy back to 40%

Now best batch size is 50 and learning rate is 0.00001 (epoch 40)

Increasing epochs from 40 to 70 got us to 84% accuracy (note with Adam got 84% but got 73% with SGD) on training set. Increasing to 120 epochs got 86% training accuracy. But I bet there would be overfitting. Should start using validation set. Note, 120 epochs got 86% on validation set versus 81% with 40 epochs

Adding regularization seemed to harm performance. I set lambda to 1, 0.1, 0.01 and all lowered accuracy. Then set lambda=10 and validation accuracy hit just under 90% on 70 epochs. Still at 90% with 120 epochs (89.6% on validation set).

In [489]:
train_labels[0:10]

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])